<a href="https://colab.research.google.com/github/moreHS/Transformer_t/blob/main/2%EC%9D%B8%EB%8C%80%ED%99%94_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import numpy as np
import tensorflow as tf
import json
import random

from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Lambda, Layer, Embedding, LayerNormalization

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
f = open('/content/drive/MyDrive/한국어 SNS/Training/[라벨]한국어SNS_train/행사.json', encoding="UTF-8")
raw_data = json.loads(f.read())


In [4]:
for x in raw_data:
  print(x)

numberOfItems
data


In [4]:
numitems=raw_data['numberOfItems']

data=raw_data['data']

In [6]:
print(np.shape(data))

(141120,)


In [5]:
print(data[0]['body'][0])
print(type(data[0]['body'][0]))

print(type(data[0]['header']['dialogueInfo']['numberOfParticipants']))

{'utteranceID': 'U1', 'turnID': 'T1', 'participantID': 'P01', 'date': '2020-07-30', 'time': '13:00:00', 'utterance': '가족분위기나 어머니 아버지가 마니 엄격하신지 조금씩 물어보구....'}
<class 'dict'>
<class 'int'>


## json 파일에서 데이터 추출

header에서는 발화자 수 추출하여 2인 대화만 저장

body에서는 발화자 id와 대화내용 저장

In [5]:
sent_lists=[]
pnum_lists=[]

for docs in data:
  sent_list=[]
  pnum_list=[]
  head=docs['header']
  docss=docs['body']
  if head['dialogueInfo']['numberOfParticipants']==2:
    for sents in docss:
      pnum = int(sents['participantID'][-1])
      sentens = sents['utterance']
      sent_list.append(sentens)
      pnum_list.append(pnum)
    sent_lists.append(sent_list)
    pnum_lists.append(pnum_list)



In [6]:
print(pnum_lists[0])
print(sent_lists[0])

[1, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 1, 2, 2, 1, 1, 2, 1]
['가족분위기나 어머니 아버지가 마니 엄격하신지 조금씩 물어보구....', 'ㅋㅋㅋㅋ웅', '얘기는 잘 통해?', '응 뭐 얘기는 괜찮아', '언제 또봐?', '몰라 아직ㅋㅋㅋ 오늘 나 필테 취소돼서 오늘 또 보잿는데 내가 걍 쉰다햇거든ㅋㅋㅋ 하루는 좀 쉬어야지 피곤', 'ㅋ. 그래.  셤셤 만나', '웅ㅋㅋㅋ', '#@시스템#사진#', '이뿌넹', '저녁에 #@이름#만나서 올거여??', '??', '갑자기 #@이름#?', '#@이름#이 오늘저녁에 집에와서 같이자고 낼 이모네 같이간다했는데', '엥 첨듣는데 ㅋㅋㅋㅋ', '연락해보께', '너랑 연락한줄알았는데 아직 안했나보네', '저녁은 어찌할지..', '웅 무러보께', '웅']


In [7]:
pip install konlpy

     |████████████████████████████████| 19.4 MB 5.3 MB/s 
     |████████████████████████████████| 448 kB 48.5 MB/s 


In [8]:
from konlpy.tag import Okt
okt=Okt()

In [9]:
print(okt.morphs(sent_lists[0][0]))
print(' '.join(okt.morphs(sent_lists[0][0])))

['가족', '분위기', '나', '어머니', '아버지', '가', '마니', '엄격하신지', '조금씩', '물어보구', '....']
가족 분위기 나 어머니 아버지 가 마니 엄격하신지 조금씩 물어보구 ....


## 발화자 (pnum)에 따라서 대화 구분, 발화자 변경 시 다음 행으로 텍스트 저장
리스트 행에 따라 발화자 구분 (발화자 무조건 교대로 저장되도록)

In [10]:
pre=1
temp_seq=[]
seq=[]

for i,lines in enumerate(sent_lists):
  for j,line in enumerate(lines):
    if pnum_lists[i][j]==pre:
      pre=pnum_lists[i][j]
      temp_seq.extend(okt.morphs(line))
    else:
      pre=pnum_lists[i][j]
      seq.append(' '.join(temp_seq))

      temp_seq=okt.morphs(line)
      



In [11]:
sent_len=[]

for x in seq:
  sent_len.append(len(x))

print(max(sent_len))
print(min(sent_len))
print(sum(sent_len)/len(sent_len))

1757
1
24.769140802110382


## 정리한 대화를 순서 한칸씩 밀어서 질문,대답으로 저장

In [13]:
print(len(seq))

questions = seq[::1][:-1]
answers = ['\t ' + lines for lines in seq[1::1]]

print(np.shape(questions))
print(np.shape(answers))

num_sample = len(questions)
num_word = 10000

970440
(970439,)
(970439,)


## Train, Test Set 분리

In [14]:

perm = list(range(num_sample))
random.seed(0)
random.shuffle(perm)

train_q = list()
train_a = list()
test_q = list()
test_a = list()

for idx, qna in enumerate(zip(questions, answers)):
    q, a = qna
    if perm[idx] > num_sample//5:
        train_q.append(q)
        train_a.append(a)
    else:
        test_q.append(q)
        test_a.append(a)

In [15]:
# 토크나이저 생성 (num_works는 빈도수 기준으로 몇개 단어까지 쓸지,)
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=num_word,
                                                  filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~ㅋ',oov_token='<OOV>')

# 텍스트의 단어들 voca로 만들기
tokenizer.fit_on_texts(train_q + train_a)
word_dict = tokenizer.word_index
print(word_dict)

# voca와 매칭되는 단어 인덱스 리스트로 만들기
train_q_seq = tokenizer.texts_to_sequences(train_q)
train_a_seq = tokenizer.texts_to_sequences(train_a)

test_q_seq = tokenizer.texts_to_sequences(test_q)
test_a_seq = tokenizer.texts_to_sequences(test_a)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [28]:
print(train_q[5])
print(train_q_seq[5])

print(train_a[5])
print(train_a_seq[5])

ㅋ . 그래 . 셤셤 만나
[106, 1, 472]
	 웅 ㅋㅋㅋ #@ 시스템 # 사진 #
[2, 26, 33, 41]


In [27]:
x_train = tf.keras.preprocessing.sequence.pad_sequences(train_q_seq, value=0, padding='pre', maxlen=80)
y_train = tf.keras.preprocessing.sequence.pad_sequences(train_a_seq, value=0, padding='post', maxlen=81)

y_train_shifted = np.concatenate([np.zeros((y_train.shape[0], 1)), y_train[:, 1:]], axis=1)

x_test = tf.keras.preprocessing.sequence.pad_sequences(test_q_seq, value=0, padding='pre', maxlen=80)
y_test = tf.keras.preprocessing.sequence.pad_sequences(test_a_seq, value=0, padding='post', maxlen=81)

# 모델 구성 정의

In [30]:
class DotScaledAttention(Layer):
    def __init__(self, d_emb, d_reduced, masked=False):
        super().__init__()
        self.q = Dense(d_reduced, input_shape=(-1, d_emb))
        self.k = Dense(d_reduced, input_shape=(-1, d_emb))
        self.v = Dense(d_reduced, input_shape=(-1, d_emb))
        self.scale = Lambda(lambda x: x/np.sqrt(d_reduced))
        self.masked = masked

    def call(self, x, training=None, mask=None): # (q,k,v)
        q = self.scale(self.q(x[0]))
        k = self.k(x[1])
        v = self.v(x[2])
        
        k_T = tf.transpose(k, perm=[0, 2, 1])
        comp = tf.matmul(q, k_T)
        
        if self.masked: # Referred from https://github.com/LastRemote/Transformer-TF2.0
            length = tf.shape(comp)[-1]
            mask = tf.fill((length, length), -np.inf)
            mask = tf.linalg.band_part(mask, 0, -1) # Get upper triangle
            mask = tf.linalg.set_diag(mask, tf.zeros((length))) # Set diagonal to zeros to avoid operations with infinity
            comp += mask
        comp = tf.nn.softmax(comp, axis=-1)
        return tf.matmul(comp, v)

In [31]:
class MultiHeadAttention(Layer):
    def __init__(self, num_head, d_emb, d_reduced, masked=False):
        super().__init__()
        self.attention_list = list()
        for _ in range(num_head):
            self.attention_list.append(DotScaledAttention(d_emb, d_reduced, masked))
        self.linear = Dense(d_emb, input_shape=(-1, num_head * d_reduced))

    def call(self, x, training=None, mask=None):
        attention_list = [a(x) for a in self.attention_list]
        concat = tf.concat(attention_list, axis=-1)
        return self.linear(concat)

In [32]:
class Encoder(Layer):
    def __init__(self, num_head, d_reduced):
        super().__init__()
        self.num_head = num_head
        self.d_r = d_reduced
    
    def build(self, input_shape):
        self.multi_attention = MultiHeadAttention(self.num_head, input_shape[-1], self.d_r)
        self.layer_norm1 = LayerNormalization(input_shape=input_shape)
        self.dense1 = Dense(input_shape[-1] * 4, input_shape=input_shape, activation='relu')
        self.dense2 = Dense(input_shape[-1],
                            input_shape=self.dense1.compute_output_shape(input_shape))
        self.layer_norm2 = LayerNormalization(input_shape=input_shape)
        super().build(input_shape)

    def call(self, x, training=None, mask=None):
        h = self.multi_attention((x, x, x))
        ln1 = self.layer_norm1(x + h)
        
        h = self.dense2(self.dense1(ln1))
        return self.layer_norm2(h + ln1)

    def compute_output_shape(self, input_shape):
        return input_shape

In [33]:
class Decoder(Layer):
    def __init__(self, num_head, d_reduced):
        super().__init__()
        self.num_head = num_head
        self.d_r = d_reduced
    
    def build(self, input_shape):
        self.self_attention = MultiHeadAttention(self.num_head, input_shape[0][-1], self.d_r, masked=True)
        self.layer_norm1 = LayerNormalization(input_shape=input_shape)
        
        self.multi_attention = MultiHeadAttention(self.num_head, input_shape[0][-1], self.d_r)
        self.layer_norm2 = LayerNormalization(input_shape=input_shape)
        
        self.dense1 = Dense(input_shape[0][-1] * 4, input_shape=input_shape[0], activation='relu')
        self.dense2 = Dense(input_shape[0][-1],
                            input_shape=self.dense1.compute_output_shape(input_shape[0]))
        self.layer_norm3 = LayerNormalization(input_shape=input_shape)
        super().build(input_shape)

    def call(self, inputs, training=None, mask=None): # (x, context)
        x, context = inputs
        h = self.self_attention((x, x, x))
        ln1 = self.layer_norm1(x + h)
        
        h = self.multi_attention((ln1, context, context))
        ln2 = self.layer_norm2(ln1 + h)
        
        h = self.dense2(self.dense1(ln2))
        return self.layer_norm3(h + ln2)

    def compute_output_shape(self, input_shape):
        return input_shape

In [34]:
class PositionalEncoding(Layer):  # Referred from https://github.com/LastRemote/Transformer-TF2.0
    def __init__(self, max_len, d_emb):
        super().__init__()
        self.sinusoidal_encoding = np.array([self.get_positional_angle(pos, d_emb) for pos in range(max_len)], dtype=np.float32)
        self.sinusoidal_encoding[:, 0::2] = np.sin(self.sinusoidal_encoding[:, 0::2])
        self.sinusoidal_encoding[:, 1::2] = np.cos(self.sinusoidal_encoding[:, 1::2])
        self.sinusoidal_encoding = tf.cast(self.sinusoidal_encoding, dtype=tf.float32)

    def call(self, x, training=None, mask=None):
        return x + self.sinusoidal_encoding[:tf.shape(x)[1]]

    def compute_output_shape(self, input_shape):
        return input_shape

    def get_angle(self, pos, dim, d_emb):
        return pos / np.power(10000, 2 * (dim // 2) / d_emb)

    def get_positional_angle(self, pos, d_emb):
        return [self.get_angle(pos, dim, d_emb) for dim in range(d_emb)]

In [35]:
class Transformer(Model):
    def __init__(self, src_vocab, dst_vocab, max_len, d_emb,
                       d_reduced, n_enc_layer, n_dec_layer, num_head):
        super().__init__()
        self.enc_emb = Embedding(src_vocab, d_emb)
        self.dec_emb = Embedding(dst_vocab, d_emb)
        self.pos_enc = PositionalEncoding(max_len, d_emb)
        
        self.encoder = [Encoder(num_head, d_reduced) for _ in range(n_enc_layer)]
        self.decoder = [Decoder(num_head, d_reduced) for _ in range(n_dec_layer)]
        self.dense = Dense(dst_vocab, input_shape=(-1, d_emb))

    def call(self, inputs, training=None, mask=None): # (src_sentence, dst_sentence_shifted)
        src_sent, dst_sent_shifted = inputs
        
        h_enc = self.pos_enc(self.enc_emb(src_sent))
        for enc in self.encoder:
            h_enc = enc(h_enc)
            
        h_dec = self.pos_enc(self.dec_emb(dst_sent_shifted))
        for dec in self.decoder:
            h_dec = dec([h_dec, h_enc])
            
        return tf.nn.softmax(self.dense(h_dec), axis=-1)

# 학습 시작

In [1]:
EPOCHS=200

transformer = Transformer(num_word,
                          num_word,
                          128, 16, 16, 2, 2, 4) # Instantiating a new transformer model

transformer.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
transformer.fit([x_train, y_train_shifted], y_train, batch_size=20, epochs=EPOCHS)

NameError: ignored

In [ ]:
import os

save_dir='/content/drive/MyDrive'

model_json = transformer.to_json()  # model.json
# with open(save_dir + "model.json", "w") as json_file:
with open(os.path.join(save_dir, "people2_conversation.json"), "w") as json_file:
    json_file.write(model_json)
transformer.save_weights(os.path.join(save_dir, "people2_conversation.h5"))  # model.h5